# 🏙️ Post 핫플레이스 하이브리드 모델

> MZ세대 유입 잠재력이 높은 행정동을 발굴하는 하이브리드 모델

**모델 구성:**
- Model 2: 서울 전체 데이터 학습 (70%)
- 핫플레이스 모델: 성수/을지로/송파/문래 데이터 학습 (30%)

## STEP 0: 데이터 로드 및 전처리

In [1]:
import pandas as pd
import glob
import numpy as np
import os
import unicodedata
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

# [환경설정] 절대 경로 설정
base_path = '/teamspace/studios/this_studio'
data_path = os.path.join(base_path, 'raw_data')

# 파일명 찾기 함수 (유니코드 정규화로 해결)
def find_file(directory, keyword):
    keyword_nfc = unicodedata.normalize('NFC', keyword)
    for f in os.listdir(directory):
        f_nfc = unicodedata.normalize('NFC', f)
        if keyword_nfc in f_nfc:
            return os.path.join(directory, f)
    return None

# 행정동명 정규화 함수 (인코딩 문제 해결)
def normalize_dong_name(name):
    """가운데점(·) 등 특수문자 인코딩 문제 해결"""
    if pd.isna(name):
        return name
    # 깨진 문자(?) 를 가운데점(·)으로 복원
    name = str(name).replace('?', '·')
    # 유니코드 정규화
    name = unicodedata.normalize('NFC', name)
    return name

# 1. 선생님이 확정한 업종 및 컬럼 리스트
target_sectors = [
    '섬유제품', '완구', '운동/경기_용품', '화장품', '문구', '서적', 
    '시계및귀금속', '안경', '일반의류', '편의점', '노래방', '미용실', 
    '당구장', '커피-음료', '호프-간이주점', '분식전문점', '치킨전문점', 
    '패스트푸드점', '제과점', '양식음식점', '중식음식점', '한식음식점'
]

cols_to_keep = [
    '기준_년분기_코드', '행정동_코드', '행정동_코드_명', '서비스_업종_코드_명', 
    '당월_매출_금액', '주중_매출_금액', '주말_매출_금액', 
    '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액'
]

# 2. 연도별 매출 데이터 통합
print("진행 중: 연도별 추정매출 통합...")
sales_files = sorted(glob.glob(os.path.join(data_path, '매출_*.csv')))
sales_list = []

for f in sales_files:
    df = pd.read_csv(f, usecols=cols_to_keep, encoding='cp949')
    df = df[df['서비스_업종_코드_명'].isin(target_sectors)]
    # 행정동명 정규화 적용
    df['행정동_코드_명'] = df['행정동_코드_명'].apply(normalize_dong_name)
    sales_list.append(df)

df_sales = pd.concat(sales_list, ignore_index=True)

# 3. 주말 및 MZ 매출 가중치 부여
print("진행 중: 주말 및 MZ 매출 가중치 적용...")
df_sales['주말_매출_가중'] = df_sales['주말_매출_금액'] * 1.5
df_sales['MZ_매출_가중'] = (
    df_sales['연령대_10_매출_금액'] * 1.2 + 
    df_sales['연령대_20_매출_금액'] * 1.2 + 
    df_sales['연령대_30_매출_금액'] * 1.2
)

df_sales['가중_총매출'] = (
    df_sales['당월_매출_금액'] * 0.3 +
    df_sales['주말_매출_가중'] * 0.3 +
    df_sales['MZ_매출_가중'] * 0.4
)

# 4. 단일 파일 데이터 로드
print("진행 중: 기타 테이블 병합...")

df_pop = pd.read_csv(os.path.join(data_path, '유동인구.csv'), encoding='cp949')
df_resident = pd.read_csv(os.path.join(data_path, '상주인구.csv'), encoding='cp949')
df_change = pd.read_csv(os.path.join(data_path, '상권변화지표.csv'), encoding='cp949')
df_facility = pd.read_csv(os.path.join(data_path, '집객시설.csv'), encoding='cp949')

# ============================================================
# 5. 카페 밀집도 변수 추가
# ============================================================
print("진행 중: 카페 밀집도 변수 생성...")

# 카페 데이터 로드 (유니코드 정규화 사용)
cafe_file = find_file(data_path, '카페')
df_cafe = pd.read_csv(cafe_file, encoding='utf-8-sig')
# 행정동_코드로 병합하기 위해 행정동_코드 포함
df_cafe = df_cafe[['기준_년분기_코드', '행정동_코드', '점포_수']]
df_cafe = df_cafe.rename(columns={'점포_수': '카페_점포_수'})

# 면적 데이터 로드 (유니코드 정규화 사용)
area_file = find_file(data_path, '면적')
df_area = pd.read_csv(area_file, encoding='cp949')
df_area = df_area[['행정동_코드', '영역_면적']]
df_area = df_area.rename(columns={'영역_면적': '행정동_면적'})

# 면적 데이터는 분기별로 안 바뀌므로 행정동별 고유값만 추출
df_area = df_area.drop_duplicates(subset='행정동_코드')

# 카페 데이터에 면적 병합 (행정동_코드로)
df_cafe = df_cafe.merge(df_area, on='행정동_코드', how='left')

# 카페 밀집도 계산 (카페 수 / 면적 km²)
df_cafe['카페_밀집도'] = df_cafe['카페_점포_수'] / (df_cafe['행정동_면적'] / 1_000_000)

print(f"  ✅ 카페 밀집도 계산 완료")
# ============================================================

# 유동인구
df_pop = df_pop[['기준_년분기_코드', '행정동_코드', '총_유동인구_수', '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수']]
df_pop['MZ_유동인구'] = df_pop['연령대_10_유동인구_수'] + df_pop['연령대_20_유동인구_수'] + df_pop['연령대_30_유동인구_수']

# 상주인구
df_resident = df_resident[['기준_년분기_코드', '행정동_코드', '총_상주인구_수', '총_가구_수']]

# 집객시설
df_facility = df_facility[['기준_년분기_코드', '행정동_코드', '집객시설_수', '지하철_역_수']]

# 상권변화지표 (운영_영업_개월_평균만 사용)
df_change = df_change[['기준_년분기_코드', '행정동_코드', '운영_영업_개월_평균']]

# 6. 최종 Merge
final_df = df_sales.merge(df_pop, on=['기준_년분기_코드', '행정동_코드'], how='left')
final_df = final_df.merge(df_resident, on=['기준_년분기_코드', '행정동_코드'], how='left')
final_df = final_df.merge(df_change, on=['기준_년분기_코드', '행정동_코드'], how='left')
final_df = final_df.merge(df_facility, on=['기준_년분기_코드', '행정동_코드'], how='left')

# 카페 밀집도 병합 (행정동_코드로 병합 - 중복 문제 해결)
final_df = final_df.merge(
    df_cafe[['기준_년분기_코드', '행정동_코드', '카페_점포_수', '카페_밀집도']], 
    on=['기준_년분기_코드', '행정동_코드'], 
    how='left'
)

final_df.fillna(0, inplace=True)

print(f"\n✅ 데이터 준비 완료! 총 {len(final_df):,}개 행")
print(f"\n추가된 변수:")
print(f"  - 카페_점포_수: 행정동별 카페 수")
print(f"  - 카페_밀집도: 카페_점포_수 / 행정동_면적(km²)")

# 행정동명 정규화 확인
unique_dongs = final_df['행정동_코드_명'].unique()
jongro_dongs = [d for d in unique_dongs if '종로' in str(d)]
print(f"\n✅ 종로 관련 행정동 확인: {jongro_dongs}")

진행 중: 연도별 추정매출 통합...
진행 중: 주말 및 MZ 매출 가중치 적용...
진행 중: 기타 테이블 병합...
진행 중: 카페 밀집도 변수 생성...
  ✅ 카페 밀집도 계산 완료

✅ 데이터 준비 완료! 총 188,294개 행

추가된 변수:
  - 카페_점포_수: 행정동별 카페 수
  - 카페_밀집도: 카페_점포_수 / 행정동_면적(km²)

✅ 종로 관련 행정동 확인: ['종로1·2·3·4가동', '종로5·6가동']


## STEP 1: 데이터 검증

In [2]:
print(f"매출 데이터 원본 행 수: {len(df_sales):,}")
print(f"최종 병합 데이터 행 수: {len(final_df):,}")

if len(df_sales) == len(final_df):
    print("✅ 성공: 데이터 누락이나 중복 없이 완벽하게 병합되었습니다.")
else:
    print("⚠️ 주의: 행 개수가 다릅니다.")

# 카페 밀집도 통계 확인
print(f"\n카페 밀집도 통계:")
print(final_df['카페_밀집도'].describe())

매출 데이터 원본 행 수: 188,294
최종 병합 데이터 행 수: 188,294
✅ 성공: 데이터 누락이나 중복 없이 완벽하게 병합되었습니다.

카페 밀집도 통계:
count    188294.000000
mean         45.298400
std          42.704756
min           0.000000
25%          19.536552
50%          35.119799
75%          58.435736
max         508.156766
Name: 카페_밀집도, dtype: float64


## STEP 2: 행정동별 그룹화 + 파생변수 생성

In [3]:
print("=" * 60)
print("[STEP 2] 행정동별 그룹화 + 파생변수 생성")
print("=" * 60)

# 행정동 + 분기별 집계
df_grouped = final_df.groupby(['기준_년분기_코드', '행정동_코드', '행정동_코드_명']).agg({
    '가중_총매출': 'sum',
    '당월_매출_금액': 'sum',
    '주말_매출_금액': 'sum',
    '총_유동인구_수': 'first',
    'MZ_유동인구': 'first',
    '총_상주인구_수': 'first',
    '집객시설_수': 'first',
    '지하철_역_수': 'first',
    '운영_영업_개월_평균': 'first',
    '카페_점포_수': 'first',
    '카페_밀집도': 'first'
}).reset_index()

# 파생변수 생성
df_grouped['상권_에너지_지수'] = df_grouped['가중_총매출'] * df_grouped['집객시설_수']
df_grouped['상권_유입_강도'] = df_grouped['총_유동인구_수'] / (df_grouped['총_상주인구_수'] + 1)
df_grouped['주말_매출_비중'] = df_grouped['주말_매출_금액'] / (df_grouped['당월_매출_금액'] + 1)

# MZ 유동인구 증가율 계산
df_grouped = df_grouped.sort_values(['행정동_코드', '기준_년분기_코드'])
df_grouped['MZ_유동_증가율'] = df_grouped.groupby('행정동_코드')['MZ_유동인구'].pct_change()
df_grouped['MZ_유동_증가율'] = df_grouped['MZ_유동_증가율'].fillna(0)
df_grouped['MZ_유동_증가율'] = df_grouped['MZ_유동_증가율'].replace([np.inf, -np.inf], 0)

print(f"\n✅ 그룹화 완료: {len(df_grouped):,}개 행")
print(f"\n생성된 파생변수:")
print(f"  - 상권_에너지_지수: 가중_총매출 × 집객시설_수")
print(f"  - 상권_유입_강도: 총_유동인구_수 / (총_상주인구_수 + 1)")
print(f"  - 주말_매출_비중: 주말_매출_금액 / 당월_매출_금액")
print(f"  - MZ_유동_증가율: MZ 유동인구 분기별 증가율")
print(f"  - 카페_밀집도: 카페_점포_수 / 행정동_면적(km²)")

[STEP 2] 행정동별 그룹화 + 파생변수 생성

✅ 그룹화 완료: 11,443개 행

생성된 파생변수:
  - 상권_에너지_지수: 가중_총매출 × 집객시설_수
  - 상권_유입_강도: 총_유동인구_수 / (총_상주인구_수 + 1)
  - 주말_매출_비중: 주말_매출_금액 / 당월_매출_금액
  - MZ_유동_증가율: MZ 유동인구 분기별 증가율
  - 카페_밀집도: 카페_점포_수 / 행정동_면적(km²)


## STEP 3: Model 2 - 서울 전체 데이터 회귀분석

In [4]:
print("=" * 60)
print("[STEP 3] Model 2: 서울 전체 데이터 회귀분석")
print("=" * 60)

# 분석용 변수 (Model 2) - 유의하지 않은 변수 제거: MZ_매출_비중, 지하철_역_수
cols_model2 = ['집객시설_수', '운영_영업_개월_평균', '상권_에너지_지수', '주말_매출_비중']

# 분석용 변수 (핫플레이스) - 유의하지 않은 변수 제거: 주말_매출_비중, 지하철_역_수
cols_hotplace = ['집객시설_수', '운영_영업_개월_평균', '상권_에너지_지수', '상권_유입_강도', '카페_밀집도']

# 전체 변수 통합 (정규화용)
cols_all = list(set(cols_model2 + cols_hotplace))

# 정규화
scaler = MinMaxScaler()
df_scaled = df_grouped.copy()
df_scaled[cols_all] = scaler.fit_transform(df_grouped[cols_all])

# Model 2 OLS 회귀분석
X_model2 = sm.add_constant(df_scaled[cols_model2])
Y_model2 = df_scaled['MZ_유동_증가율']

model2 = sm.OLS(Y_model2, X_model2).fit()
model2_weights = model2.params

print("\n=== Model 2 회귀분석 결과 ===")
print(model2.summary())
print("\n★ Model 2 가중치:")
print(model2_weights)

[STEP 3] Model 2: 서울 전체 데이터 회귀분석

=== Model 2 회귀분석 결과 ===
                            OLS Regression Results                            
Dep. Variable:              MZ_유동_증가율   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     5.991
Date:                Tue, 27 Jan 2026   Prob (F-statistic):           8.20e-05
Time:                        09:04:25   Log-Likelihood:                 17583.
No. Observations:               11443   AIC:                        -3.516e+04
Df Residuals:                   11438   BIC:                        -3.512e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

## STEP 4: 핫플레이스 통합 모델

In [5]:
print("=" * 60)
print("[STEP 4] 핫플레이스 통합 모델 (성수/을지로/송파/문래)")
print("=" * 60)

# 대상 핫플레이스 행정동 리스트
target_hotplaces = ['성수', '을지로동', '송파1동', '송파2동', '문래동']

# 통합 데이터 필터링
hotplace_df = df_scaled[df_scaled['행정동_코드_명'].apply(
    lambda x: any(hp in x for hp in target_hotplaces)
)].copy()

print(f"\n학습 데이터: {len(hotplace_df)}개")
print(f"포함 행정동: {hotplace_df['행정동_코드_명'].unique()}")

# 핫플레이스 회귀분석
X_hotplace = sm.add_constant(hotplace_df[cols_hotplace])
Y_hotplace = hotplace_df['MZ_유동_증가율']

model_hotplace = sm.OLS(Y_hotplace, X_hotplace).fit()
hotplace_weights = model_hotplace.params

print("\n=== 핫플레이스 회귀분석 결과 ===")
print(model_hotplace.summary())

[STEP 4] 핫플레이스 통합 모델 (성수/을지로/송파/문래)

학습 데이터: 216개
포함 행정동: ['을지로동' '성수1가1동' '성수1가2동' '성수2가1동' '성수2가3동' '문래동' '송파1동' '송파2동']

=== 핫플레이스 회귀분석 결과 ===
                            OLS Regression Results                            
Dep. Variable:              MZ_유동_증가율   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     2.566
Date:                Tue, 27 Jan 2026   Prob (F-statistic):             0.0281
Time:                        09:04:25   Log-Likelihood:                 371.25
No. Observations:                 216   AIC:                            -730.5
Df Residuals:                     210   BIC:                            -710.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.

## STEP 5: 하이브리드 점수 계산 (정규화 적용)

In [6]:
print("=" * 60)
print("[STEP 5] 하이브리드 점수 계산")
print("=" * 60)

# Model 2 점수
df_scaled['Model2_점수'] = (
    df_scaled['상권_에너지_지수'] * model2_weights['상권_에너지_지수'] +
    df_scaled['운영_영업_개월_평균'] * model2_weights['운영_영업_개월_평균'] +
    df_scaled['집객시설_수'] * model2_weights['집객시설_수'] +
    df_scaled['주말_매출_비중'] * model2_weights['주말_매출_비중']
)

# 핫플레이스 유사도 점수
df_scaled['핫플_유사도_점수'] = (
    hotplace_weights['const'] +
    df_scaled['집객시설_수'] * hotplace_weights['집객시설_수'] +
    df_scaled['운영_영업_개월_평균'] * hotplace_weights['운영_영업_개월_평균'] +
    df_scaled['상권_에너지_지수'] * hotplace_weights['상권_에너지_지수'] +
    df_scaled['상권_유입_강도'] * hotplace_weights['상권_유입_강도'] +
    df_scaled['카페_밀집도'] * hotplace_weights['카페_밀집도']
)

# 점수 정규화
scaler_final = MinMaxScaler()
df_scaled[['Model2_점수_정규화', '핫플_유사도_정규화']] = scaler_final.fit_transform(
    df_scaled[['Model2_점수', '핫플_유사도_점수']]
)

# 하이브리드 점수
WEIGHT_MODEL2 = 0.7
WEIGHT_HOTPLACE = 0.3

df_scaled['하이브리드_점수'] = (
    df_scaled['Model2_점수_정규화'] * WEIGHT_MODEL2 + 
    df_scaled['핫플_유사도_정규화'] * WEIGHT_HOTPLACE
)

print(f"\n가중치: Model 2 ({WEIGHT_MODEL2*100:.0f}%) + 핫플 유사도 ({WEIGHT_HOTPLACE*100:.0f}%)")
print(f"\n점수 범위:")
print(f"  - Model2_점수: {df_scaled['Model2_점수'].min():.4f} ~ {df_scaled['Model2_점수'].max():.4f}")
print(f"  - 핫플_유사도_점수: {df_scaled['핫플_유사도_점수'].min():.4f} ~ {df_scaled['핫플_유사도_점수'].max():.4f}")
print(f"  - 하이브리드_점수: {df_scaled['하이브리드_점수'].min():.4f} ~ {df_scaled['하이브리드_점수'].max():.4f}")

[STEP 5] 하이브리드 점수 계산

가중치: Model 2 (70%) + 핫플 유사도 (30%)

점수 범위:
  - Model2_점수: -0.0197 ~ 0.0149
  - 핫플_유사도_점수: -0.0621 ~ 145.9887
  - 하이브리드_점수: 0.0001 ~ 0.7021


## STEP 6: 최종 랭킹

In [ ]:
print("=" * 60)
print("[STEP 6] Post 핫플레이스 최종 랭킹")
print("=" * 60)

# 행정동별 평균 점수 계산 
ranking_hybrid = df_scaled.groupby('행정동_코드_명').agg({
    'Model2_점수': 'mean',
    '핫플_유사도_점수': 'mean',
    '하이브리드_점수': 'mean',
    '상권_유입_강도': 'mean',
    '주말_매출_비중': 'mean',
    '카페_밀집도': 'mean',
}).reset_index()

# 전체 랭킹 (핫플레이스 포함)
ranking_all = ranking_hybrid.sort_values('하이브리드_점수', ascending=False).reset_index(drop=True)
ranking_all.insert(0, 'Rank', range(1, len(ranking_all) + 1))

# 학습에 사용한 핫플레이스 리스트 (제외 대상)
exclude_hotplaces = ['성수', '을지로동', '송파1동', '송파2동', '문래동']

# 현재 핫플레이스 순위 확인 (검증용)
print("\n★ 현재 핫플레이스 순위 (검증용):")
hotplace_rank = ranking_all[ranking_all['행정동_코드_명'].apply(
    lambda x: any(hp in x for hp in exclude_hotplaces)
)]
print(hotplace_rank[['Rank', '행정동_코드_명', '하이브리드_점수', 'Model2_점수', '핫플_유사도_점수', '카페_밀집도']].to_string(index=False))

ranking_all['Rank'] = range(1, len(ranking_all) + 1)

print("\n" + "=" * 60)
print("★★★ Post 핫플레이스 TOP 30 ★★★")
print("=" * 60)
print(ranking_all[['Rank', '행정동_코드_명', '하이브리드_점수', 'Model2_점수', '핫플_유사도_점수', '카페_밀집도']].head(30).to_string(index=False))

[STEP 6] Post 핫플레이스 최종 랭킹

★ 현재 핫플레이스 순위 (검증용):
 Rank 행정동_코드_명  하이브리드_점수  Model2_점수  핫플_유사도_점수   카페_밀집도
   15   성수2가3동  0.227174  -0.008473   0.014390 0.243987
   28   성수1가2동  0.216055  -0.009021   0.006413 0.340804
   46     송파2동  0.197749  -0.009923  -0.007280 0.090323
   97   성수1가1동  0.180532  -0.010773   0.000571 0.056928
   98      문래동  0.180342  -0.010783   0.002015 0.104389
  108   성수2가1동  0.177679  -0.010914   0.004944 0.127015
  137     송파1동  0.171906  -0.011198  -0.008458 0.364147
  169     을지로동  0.166871  -0.011448   0.008727 0.321921

★★★ Post 핫플레이스 TOP 30 ★★★
 Rank    행정동_코드_명  하이브리드_점수  Model2_점수  핫플_유사도_점수   카페_밀집도
    1         가산동  0.518814   0.005855   0.654381 0.145402
    2        역삼1동  0.417960   0.000903   0.409285 0.310161
    3 종로1·2·3·4가동  0.406832   0.000344   0.506696 0.307772
    4         서교동  0.326954  -0.003577   0.299196 0.688097
    5         여의동  0.279768  -0.005892   0.157597 0.056165
    6        둔촌1동  0.266820  -0.006520   0.055761 0.007191
    7   

## STEP 7: 결과 저장

In [8]:
# CSV 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f'Post_핫플레이스_하이브리드_랭킹_{timestamp}.csv'
output_path = os.path.join(base_path, output_filename)

# 저장할 컬럼 선택
save_cols = ['Rank', '행정동_코드_명', '하이브리드_점수', 'Model2_점수', '핫플_유사도_점수', '상권_유입_강도', '주말_매출_비중', '카페_밀집도']
ranking_all[save_cols].head(50).to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n✅ 결과 저장 완료: {output_path}")
print(f"   저장된 행정동 수: 50개")


✅ 결과 저장 완료: /teamspace/studios/this_studio/Post_핫플레이스_하이브리드_랭킹_20260127_090425.csv
   저장된 행정동 수: 50개


## STEP 8: 분석 요약 리포트

In [9]:
print("=" * 60)
print("[분석 요약 리포트]")
print("=" * 60)

print("""
★ 분석 방법론:

1. Model 2 (서울 전체 학습)
   - 데이터: 서울 전체 행정동 (11,443개 관측치)
   - 변수: 집객시설_수, 운영_영업_개월_평균, 상권_에너지_지수, 주말_매출_비중

2. 핫플레이스 통합 모델
   - 데이터: 성수동(4개동) + 을지로동 + 송파1동 + 송파2동 + 문래동 (216개 관측치)
   - 변수: 집객시설_수, 운영_영업_개월_평균, 상권_에너지_지수, 상권_유입_강도, 카페_밀집도

3. 하이브리드 점수
   - 구성: Model 2 (70%) + 핫플 유사도 (30%)
   - 정규화: MinMaxScaler (0~1 범위)

★ Post 핫플레이스 선정 기준:
   - 하이브리드 점수가 높은 지역
   - 기존 핫플레이스와 유사한 특성 보유
   - MZ 성장 잠재력이 있는 지역
""")

# TOP 10 상세 분석
print("=" * 60)
print("★ Post 핫플레이스 TOP 10")
print("=" * 60)
top10 = ranking_all.head(10)
for idx, row in top10.iterrows():
    print(f"\n{int(row['Rank'])}위: {row['행정동_코드_명']}")
    print(f"   - 하이브리드 점수: {row['하이브리드_점수']:.4f}")
    print(f"   - Model2 점수: {row['Model2_점수']:.4f}")
    print(f"   - 핫플 유사도: {row['핫플_유사도_점수']:.4f}")
    print(f"   - 카페 밀집도: {row['카페_밀집도']:.2f}")

[분석 요약 리포트]

★ 분석 방법론:

1. Model 2 (서울 전체 학습)
   - 데이터: 서울 전체 행정동 (11,443개 관측치)
   - 변수: 집객시설_수, 운영_영업_개월_평균, 상권_에너지_지수, 주말_매출_비중

2. 핫플레이스 통합 모델
   - 데이터: 성수동(4개동) + 을지로동 + 송파1동 + 송파2동 + 문래동 (216개 관측치)
   - 변수: 집객시설_수, 운영_영업_개월_평균, 상권_에너지_지수, 상권_유입_강도, 카페_밀집도

3. 하이브리드 점수
   - 구성: Model 2 (70%) + 핫플 유사도 (30%)
   - 정규화: MinMaxScaler (0~1 범위)

★ Post 핫플레이스 선정 기준:
   - 하이브리드 점수가 높은 지역
   - 기존 핫플레이스와 유사한 특성 보유
   - MZ 성장 잠재력이 있는 지역

★ Post 핫플레이스 TOP 10

1위: 가산동
   - 하이브리드 점수: 0.5188
   - Model2 점수: 0.0059
   - 핫플 유사도: 0.6544
   - 카페 밀집도: 0.15

2위: 역삼1동
   - 하이브리드 점수: 0.4180
   - Model2 점수: 0.0009
   - 핫플 유사도: 0.4093
   - 카페 밀집도: 0.31

3위: 종로1·2·3·4가동
   - 하이브리드 점수: 0.4068
   - Model2 점수: 0.0003
   - 핫플 유사도: 0.5067
   - 카페 밀집도: 0.31

4위: 서교동
   - 하이브리드 점수: 0.3270
   - Model2 점수: -0.0036
   - 핫플 유사도: 0.2992
   - 카페 밀집도: 0.69

5위: 여의동
   - 하이브리드 점수: 0.2798
   - Model2 점수: -0.0059
   - 핫플 유사도: 0.1576
   - 카페 밀집도: 0.06

6위: 둔촌1동
   - 하이브리드 점수: 0.2668
   - Model2 점수: -0.0065
   - 핫플 유사도: 0.0558